In [6]:
import pandas as pd
from sklearn_pandas import DataFrameMapper
import numpy as np
import seaborn as sns
import matplotlib
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelBinarizer, LabelEncoder, Imputer

In [2]:
df = pd.read_csv("movie_metadata.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 28 columns):
color                        5024 non-null object
director_name                4939 non-null object
num_critic_for_reviews       4993 non-null float64
duration                     5028 non-null float64
director_facebook_likes      4939 non-null float64
actor_3_facebook_likes       5020 non-null float64
actor_2_name                 5030 non-null object
actor_1_facebook_likes       5036 non-null float64
gross                        4159 non-null float64
genres                       5043 non-null object
actor_1_name                 5036 non-null object
movie_title                  5043 non-null object
num_voted_users              5043 non-null int64
cast_total_facebook_likes    5043 non-null int64
actor_3_name                 5020 non-null object
facenumber_in_poster         5030 non-null float64
plot_keywords                4890 non-null object
movie_imdb_link              5043 non-

In [3]:
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [4]:
df.isnull().any()

color                         True
director_name                 True
num_critic_for_reviews        True
duration                      True
director_facebook_likes       True
actor_3_facebook_likes        True
actor_2_name                  True
actor_1_facebook_likes        True
gross                         True
genres                       False
actor_1_name                  True
movie_title                  False
num_voted_users              False
cast_total_facebook_likes    False
actor_3_name                  True
facenumber_in_poster          True
plot_keywords                 True
movie_imdb_link              False
num_user_for_reviews          True
language                      True
country                       True
content_rating                True
budget                        True
title_year                    True
actor_2_facebook_likes        True
imdb_score                   False
aspect_ratio                  True
movie_facebook_likes         False
dtype: bool

In [5]:
df_genres = df.genres.apply(lambda x: x.split('|'))

In [6]:
genres = df_genres.apply(pd.Series)

In [7]:
df_genres.to_frame().head()

,genres
0,"[Action, Adventure, Fantasy, Sci-Fi]"
1,"[Action, Adventure, Fantasy]"
2,"[Action, Adventure, Thriller]"
3,"[Action, Thriller]"
4,[Documentary]


In [8]:
df_genres.index
#df_genres.iterrows()
#df["genres"]
df.iterrows()

<generator object DataFrame.iterrows at 0x7f44c4b1df10>

In [9]:
def explode(df, df_col):
    rows = []
    for i,row in df.iterrows():
        for element in row[df_col]:
            rows.append([i,element])
    return pd.DataFrame(data=rows, columns=["old_index", df_col])

In [10]:
df_exploded = explode(df_genres.to_frame(), "genres")
#df_exploded.pivot(index=)
df_exploded.head()
df_exploded["val"] = int(1)
df_exploded.head()
df_genres_pivot = df_exploded.pivot(index="old_index", columns="genres")["val"].fillna(int(0))

df_genres_pivot.head()

genres,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
old_index,,,,,,,,,,,,,,,,,,,,,
0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df_genres_pivot.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5043 entries, 0 to 5042
Data columns (total 26 columns):
Action         5043 non-null float64
Adventure      5043 non-null float64
Animation      5043 non-null float64
Biography      5043 non-null float64
Comedy         5043 non-null float64
Crime          5043 non-null float64
Documentary    5043 non-null float64
Drama          5043 non-null float64
Family         5043 non-null float64
Fantasy        5043 non-null float64
Film-Noir      5043 non-null float64
Game-Show      5043 non-null float64
History        5043 non-null float64
Horror         5043 non-null float64
Music          5043 non-null float64
Musical        5043 non-null float64
Mystery        5043 non-null float64
News           5043 non-null float64
Reality-TV     5043 non-null float64
Romance        5043 non-null float64
Sci-Fi         5043 non-null float64
Short          5043 non-null float64
Sport          5043 non-null float64
Thriller       5043 non-null float64
War  

In [12]:
df_mod = df.join(df_genres_pivot)
df_mod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 54 columns):
color                        5024 non-null object
director_name                4939 non-null object
num_critic_for_reviews       4993 non-null float64
duration                     5028 non-null float64
director_facebook_likes      4939 non-null float64
actor_3_facebook_likes       5020 non-null float64
actor_2_name                 5030 non-null object
actor_1_facebook_likes       5036 non-null float64
gross                        4159 non-null float64
genres                       5043 non-null object
actor_1_name                 5036 non-null object
movie_title                  5043 non-null object
num_voted_users              5043 non-null int64
cast_total_facebook_likes    5043 non-null int64
actor_3_name                 5020 non-null object
facenumber_in_poster         5030 non-null float64
plot_keywords                4890 non-null object
movie_imdb_link              5043 non-

In [14]:
#imputerBla = Imputer(copy=False)

mapper = DataFrameMapper([
 (['duration', 'budget'], [Imputer(), StandardScaler()],
) 
],df_out=False)



#mapper = DataFrameMapper([
 #   ('color', LabelBinarizer()),
 #   ('director_name', LabelBinarizer()),
 #   ('budget_imputed', StandardScaler()),
 #   ('duration_imputed', StandardScaler())
#])

#pipelines = Pipeline([
#    ('imputer', imputers),
#    ('mapper', mapper)
#])

In [15]:
#mapper.fit_transform(df)

#>>> mapper3 = DataFrameMapper([
#...     (['ae'], [sklearn.preprocessing.Imputer(),
#...                sklearn.preprocessing.StandardScaler()])])
#>>> data_3 = pd.DataFrame({'age': [1, np.nan, 3]})
#>>> mapper3.fit_transform(data_3)
#array([[-1.22474487],
#       [ 0.        ],
#       [ 1.22474487]])

In [17]:
#df_imputed = imputers.transform(df)
#df_imputed

In [18]:
df.head()
mapper.fit(df)

DataFrameMapper(default=False, df_out=False,
        features=[(['duration', 'budget'], [Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0), StandardScaler(copy=True, with_mean=True, with_std=True)])],
        input_df=False, sparse=False)

In [19]:
df["duration"].head()

0    178.0
1    169.0
2    148.0
3    164.0
4      NaN
Name: duration, dtype: float64

In [21]:
#type(df_imputed)

In [23]:
#test = StandardScaler()
#test.fit(X=df_imputed.T)

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin

class SampleExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, vars):
        self.vars = vars  # e.g. pass in a column name to extract

    def transform(self, X, y=None):
        return do_something_to(X, self.vars)  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # generally does nothing

# Custom Transformer for Pipelines

In [25]:
from sklearn.base import TransformerMixin

class ArrayExplodePivot(TransformerMixin):

    def __init__(self, col_name):
        self.col_name = col_name

    def transform(self, X):
        df = self.__explode__(X)
        df_pivot = self.__pivot__(df)
        df_pivot.index.names = ["index"]
        return self.__join__(X, df_pivot)
    
    def fit(self, X):
        return self  # generally does nothing
    
    def __explode__(self, X):
        rows = []
        for i,row in X.iterrows():
            for element in row[self.col_name]:
                rows.append([i,element])
        return pd.DataFrame(data=rows, columns=["old_index", self.col_name])
    
    def __pivot__(self, X):
        X["val"] = int(1)
        return X.pivot(index="old_index", columns=self.col_name)["val"].fillna(int(0))
    
    def __join__(self, X, Y):
        return X.join(Y)

    
class StringToArray(TransformerMixin):
    ''' Takes a Pandas Dataframe and two column names as input and transforms the input column '''
    def __init__(self, col_name_in, col_name_out, seperator):
        self.col_name_in = col_name_in  # e.g. pass in a column name to extract
        self.col_name_out = col_name_out  # e.g. pass in a column name to extract
        self.seperator = seperator

    def transform(self, X):
        X[self.col_name_out] = X[self.col_name_in].apply(lambda x: x.split('|'))
        return X # where the actual feature extraction happens
    
    def __join__(self, X, Y):
        return X.join(Y)

    def fit(self, X):
        return self  # generally does nothing

In [26]:
pipeTest = Pipeline([
    ('split', StringToArray("genres", "genres_out", "|")),
    ('explode', ArrayExplodePivot("genres_out"))
])

In [27]:
df_test = df
df_new = pipeTest.transform(df_test).head()

In [28]:
# extract the step 2
pipeTest.steps[0][1].__doc__

' Takes a Pandas Dataframe and two column names as input and transforms the input column '

In [29]:
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer

In [30]:
def get_text_cols(df):
    return df[['actor1_name', 'actor_2_name']]

def get_num_cols(df):
    return df[['height','age']]


In [40]:
#%%timeit
%time
df_test.columns.values.tolist()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.7 µs


['color',
 'director_name',
 'num_critic_for_reviews',
 'duration',
 'director_facebook_likes',
 'actor_3_facebook_likes',
 'actor_2_name',
 'actor_1_facebook_likes',
 'gross',
 'genres',
 'actor_1_name',
 'movie_title',
 'num_voted_users',
 'cast_total_facebook_likes',
 'actor_3_name',
 'facenumber_in_poster',
 'plot_keywords',
 'movie_imdb_link',
 'num_user_for_reviews',
 'language',
 'country',
 'content_rating',
 'budget',
 'title_year',
 'actor_2_facebook_likes',
 'imdb_score',
 'aspect_ratio',
 'movie_facebook_likes',
 'genres_out']

In [41]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5043 entries, 0 to 5042
Data columns (total 29 columns):
color                        5024 non-null object
director_name                4939 non-null object
num_critic_for_reviews       4993 non-null float64
duration                     5028 non-null float64
director_facebook_likes      4939 non-null float64
actor_3_facebook_likes       5020 non-null float64
actor_2_name                 5030 non-null object
actor_1_facebook_likes       5036 non-null float64
gross                        4159 non-null float64
genres                       5043 non-null object
actor_1_name                 5036 non-null object
movie_title                  5043 non-null object
num_voted_users              5043 non-null int64
cast_total_facebook_likes    5043 non-null int64
actor_3_name                 5020 non-null object
facenumber_in_poster         5030 non-null float64
plot_keywords                4890 non-null object
movie_imdb_link              5043 non-

In [43]:
#vec = make_union([make_pipeline(FunctionTransformer(get_text_cols, validate=False), 
#                                LabelEncoder()))),
#                                  make_pipeline(FunctionTransformer(get_num_cols, validate=False), 
#                                MinMaxScaler())))])

SyntaxError: invalid syntax (<ipython-input-43-9d1f1e01ccb0>, line 2)

In [44]:
labelEncoder = LabelEncoder()

In [45]:
labelEncoder.fit(df_test.movie_title)

LabelEncoder()

In [46]:
labelEncoder.transform(df_test.movie_title)

array([ 396, 2725, 3274, ...,  133, 3107, 2458])

In [55]:
df_actors = df_test["actor_1_name"]

In [57]:
type(df_actors)

pandas.core.series.Series

In [63]:
type(df.actor_3_name)
type(df["actor_3_name"])

pandas.core.series.Series

In [64]:
pd.set_option('display.max_rows', 1000)

In [68]:
df.actor_2_name.head()

0    Joel David Moore
1       Orlando Bloom
2        Rory Kinnear
3      Christian Bale
4          Rob Walker
Name: actor_2_name, dtype: object

In [67]:
pd.set_option('display.notebook_repr_html', True)

In [76]:
df[["color", "genres"]]

,color,genres
0,Color,Action|Adventure|Fantasy|Sci-Fi
1,Color,Action|Adventure|Fantasy
2,Color,Action|Adventure|Thriller
3,Color,Action|Thriller
4,NaN,Documentary
5,Color,Action|Adventure|Sci-Fi
6,Color,Action|Adventure|Romance
7,Color,Adventure|Animation|Comedy|Family|Fantasy|Musi...
8,Color,Action|Adventure|Sci-Fi
9,Color,Adventure|Family|Fantasy|Mystery


In [78]:
labelBinarizer = LabelBinarizer()

In [81]:
test2 = labelBinarizer.fit(df.actor_1_name.dropna())

In [83]:
labelBinarizer.transform(df.actor_1_name.dropna())

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [87]:
df.select_dtypes(include=["object"])

,color,director_name,actor_2_name,genres,actor_1_name,movie_title,actor_3_name,plot_keywords,movie_imdb_link,language,country,content_rating,genres_out
0,Color,James Cameron,Joel David Moore,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,Wes Studi,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,English,USA,PG-13,"[Action, Adventure, Fantasy, Sci-Fi]"
1,Color,Gore Verbinski,Orlando Bloom,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,Jack Davenport,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,English,USA,PG-13,"[Action, Adventure, Fantasy]"
2,Color,Sam Mendes,Rory Kinnear,Action|Adventure|Thriller,Christoph Waltz,Spectre,Stephanie Sigman,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,English,UK,PG-13,"[Action, Adventure, Thriller]"
3,Color,Christopher Nolan,Christian Bale,Action|Thriller,Tom Hardy,The Dark Knight Rises,Joseph Gordon-Levitt,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,English,USA,PG-13,"[Action, Thriller]"
4,NaN,Doug Walker,Rob Walker,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,NaN,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,[Documentary]
5,Color,Andrew Stanton,Samantha Morton,Action|Adventure|Sci-Fi,Daryl Sabara,John Carter,Polly Walker,alien|american civil war|male nipple|mars|prin...,http://www.imdb.com/title/tt0401729/?ref_=fn_t...,English,USA,PG-13,"[Action, Adventure, Sci-Fi]"
6,Color,Sam Raimi,James Franco,Action|Adventure|Romance,J.K. Simmons,Spider-Man 3,Kirsten Dunst,sandman|spider man|symbiote|venom|villain,http://www.imdb.com/title/tt0413300/?ref_=fn_t...,English,USA,PG-13,"[Action, Adventure, Romance]"
7,Color,Nathan Greno,Donna Murphy,Adventure|Animation|Comedy|Family|Fantasy|Musi...,Brad Garrett,Tangled,M.C. Gainey,17th century|based on fairy tale|disney|flower...,http://www.imdb.com/title/tt0398286/?ref_=fn_t...,English,USA,PG,"[Adventure, Animation, Comedy, Family, Fantasy..."
8,Color,Joss Whedon,Robert Downey Jr.,Action|Adventure|Sci-Fi,Chris Hemsworth,Avengers: Age of Ultron,Scarlett Johansson,artificial intelligence|based on comic book|ca...,http://www.imdb.com/title/tt2395427/?ref_=fn_t...,English,USA,PG-13,"[Action, Adventure, Sci-Fi]"
9,Color,David Yates,Daniel Radcliffe,Adventure|Family|Fantasy|Mystery,Alan Rickman,Harry Potter and the Half-Blood Prince,Rupert Grint,blood|book|love|potion|professor,http://www.imdb.com/title/tt0417741/?ref_=fn_t...,English,UK,PG,"[Adventure, Family, Fantasy, Mystery]"


In [90]:
df.select_dtypes(include=["float"]).columns.values.tolist()

['num_critic_for_reviews',
 'duration',
 'director_facebook_likes',
 'actor_3_facebook_likes',
 'actor_1_facebook_likes',
 'gross',
 'facenumber_in_poster',
 'num_user_for_reviews',
 'budget',
 'title_year',
 'actor_2_facebook_likes',
 'imdb_score',
 'aspect_ratio']

In [97]:
myImputer = Imputer(strategy="most_frequent", axis = 1)

In [98]:
myImputer.fit(df.color)
#df.color

Imputer(axis=1, copy=True, missing_values='NaN', strategy='most_frequent',
    verbose=0)

In [102]:
class GeneralImputer(Imputer):
    def __init__(self, **kwargs):
        Imputer.__init__(self, **kwargs)

    def fit(self, X, y=None):
        if self.strategy == 'most_frequent':
            self.fills = pd.DataFrame(X).mode(axis=0).squeeze()
            self.statistics_ = self.fills.values
            return self
        else:
            return Imputer.fit(self, X, y=y)

    def transform(self, X):
        if hasattr(self, 'fills'):
            return pd.DataFrame(X).fillna(self.fills).values.astype(str)
        else:
            return Imputer.transform(self, X)

In [107]:
class DataFrameImputer(TransformerMixin):

    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
    def fit(self, X, y=None):

        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)

        return self

    def transform(self, X, y=None):
        return X.fillna(self.fill)

In [108]:
dataFrameImputer = DataFrameImputer()

In [118]:
dataFrameImputer.fit(df[["color"]])

In [119]:
dataFrameImputer.transform(df)

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,genres_out
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000,"[Action, Adventure, Fantasy, Sci-Fi]"
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0,"[Action, Adventure, Fantasy]"
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000,"[Action, Adventure, Thriller]"
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000,"[Action, Thriller]"
4,Color,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0,[Documentary]
5,Color,Andrew Stanton,462.0,132.0,475.0,530.0,Samantha Morton,640.0,73058679.0,Action|Adventure|Sci-Fi,...,English,USA,PG-13,263700000.0,2012.0,632.0,6.6,2.35,24000,"[Action, Adventure, Sci-Fi]"
6,Color,Sam Raimi,392.0,156.0,0.0,4000.0,James Franco,24000.0,336530303.0,Action|Adventure|Romance,...,English,USA,PG-13,258000000.0,2007.0,11000.0,6.2,2.35,0,"[Action, Adventure, Romance]"
7,Color,Nathan Greno,324.0,100.0,15.0,284.0,Donna Murphy,799.0,200807262.0,Adventure|Animation|Comedy|Family|Fantasy|Musi...,...,English,USA,PG,260000000.0,2010.0,553.0,7.8,1.85,29000,"[Adventure, Animation, Comedy, Family, Fantasy..."
8,Color,Joss Whedon,635.0,141.0,0.0,19000.0,Robert Downey Jr.,26000.0,458991599.0,Action|Adventure|Sci-Fi,...,English,USA,PG-13,250000000.0,2015.0,21000.0,7.5,2.35,118000,"[Action, Adventure, Sci-Fi]"
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,...,English,UK,PG,250000000.0,2009.0,11000.0,7.5,2.35,10000,"[Adventure, Family, Fantasy, Mystery]"
